In [1]:
%matplotlib ipympl
from mantid.simpleapi import *
from tkinter import *
import matplotlib.pyplot as plt
from mantid.plots.utility import MantidAxType
from matplotlib.colors import LightSource
from matplotlib import cm
import matplotlib as mpl
from mantid.api import AnalysisDataService as ADS
import functools
import scipy.io
import numpy as np
import pathlib
import math as m
from datetime import datetime as dt
from scipy.signal import find_peaks as fp
import os
import json

This function takes the run, the experiment conducted, and the bank the user wishes to retrieve the data from.

In [3]:
#This is the combined filesearch function bringing all the capabilities of the seperate functions together
def fileSearch(run,exp,bank):
    exps = ['Cryo','Furnace','Pressure','X Scan','Y Scan','Z Scan']
    if len(str(run)) < 5:
        zero_diff = 5 - len(str(run))
        for i in range(zero_diff):
            run = f'0{run}'
            
    yr = dt.now().strftime('%y')
    cy = int(yr) + 1

    if bank == '1':
        sMin_One = 0
        sMax_One = 1200
        sMin_Two = 1
        sMax_Two = 1199
    elif bank == '2':
        sMin_One = 1201
        sMax_One = 2400
        sMin_Two = 1200
        sMax_Two = 2399
    elif bank == 'Both':
        sMin_One = 0
        sMax_One = 2400
        sMin_Two = 1
        sMax_Two = 2399
    else:
        sMin_One = 0
        sMax_One = 2400
        sMin_Two = 1
        sMax_Two = 2399
    for i in reversed(range(2,int(cy))):
        if i < 10:
            year = f'0{i}'
        else:
            year = f'{i}'
        if i < 17:
            apps = [['cryo_1','cryo_temp1'],'furnace','position','x','y','z']
            if exp == exps[0]:
                if i < 10:
                    app = apps[0][0]
                else:
                    app = apps[0][1]
            elif exp == exps[1]:
                app = apps[1]
            elif exp == exps[2]:
                app = apps[2]
            elif exp == exps[3]:
                app = apps[3]
            elif exp == exps[4]:
                app = apps[4]
            elif exp == exps[5]:
                app = apps[5]
            elif exp == None:
                pass
            else:
                raise NameError(f'This is not included in the list of experiments. Please enter one from this list: {exps}')
        else:
            apps = ['Temp_1','Temp_3','position','X_position','Y_position','Z_position']
            if exp == exps[0]:
                app = apps[0]
            elif exp == exps[1]:
                app = apps[1]
            elif exp == exps[2]:
                app = apps[2]
            elif exp == exps[3]:
                app = apps[3]
            elif exp == exps[4]:
                app = apps[4]
            elif exp == exps[5]:
                app = apps[5]
            elif exp == None:
                pass
            else:
                raise NameError(f'This is not included in the list of experiments. Please enter one from this list: {exps}')
        for cycle in [5,4,3,2,1]:
            if i < 8:
                name_one = f'ENG{run}'
                name_two = f'ENG{run}'
            else:
                name_one = f'ENG00{run}'
                name_two = f'ENGINX00{run}'
            ext_one = '.raw'
            ext_two = '.nxs'
            testdir_one = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}{ext_one}'
            testdir_two = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}{ext_one}'
            testdir_three = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}{ext_two}'
            testdir_four = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}{ext_two}'
            check_one = pathlib.Path(testdir_one)
            check_two = pathlib.Path(testdir_two)
            check_three = pathlib.Path(testdir_three)
            check_four = pathlib.Path(testdir_four)
            if check_one.exists():
                dire = testdir_one
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp = np.loadtxt(fname=f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}_{app}.txt',dtype='float',usecols=(1))
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                return dire, temp, sumSpec
            elif check_two.exists():
                dire = testdir_two
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp = np.loadtxt(fname=f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}_{app}.txt',dtype='float',usecols=(1))
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                return dire, temp, sumSpec
            elif check_three.exists():
                dire = testdir_three
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp = np.loadtxt(fname=f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}_{app}.txt',dtype='float',usecols=(1))
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                return dire, temp, sumSpec
            elif check_four.exists():
                dire = testdir_four
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp = np.loadtxt(fname=f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}_{app}.txt',dtype='float',usecols=(1))
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                return dire, temp, sumSpec
            else:
                pass

In [4]:
dire,temp,sumSpec = fileSearch(346541,exp='Cryo',bank=None)
print(dire,temp,sumSpec)

//isis/inst$/NDXENGINX/Instrument/data/cycle_24_2/ENGINX00346541.raw 242.92 ENGINX00346541-sumSpec
